Train MNIST
-------------------------

In [1]:
from __future__ import print_function
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
device

'cpu'

Import from the git

In [3]:
!git clone https://github.com/kynehc/clone_anonymous_github.git
%cd clone_anonymous_github
!python3 download.py --url https://anonymous.4open.science/r/GaussNetworks-A7E7/ --dir gauss_nets/
%cd gauss_nets
!ls

fatal: destination path 'clone_anonymous_github' already exists and is not an empty directory.
/Users/shess/Scripts/clone_anonymous_github
[*] cloning project:GaussNetworks-A7E7
[*] downloading files:
[*] files saved to:gauss_nets/
/Users/shess/Scripts/clone_anonymous_github/gauss_nets
README.md  checkpoint data       train


In [4]:
import train
from train.models import *
from train import *
from train.layers import *
from train.loss import *
from train.optimization import Optimizer

Load the data

In [5]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
c=10

# Data
print('==> Preparing data..')
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=trans)
trainloader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)


testset = datasets.MNIST(root='./data', train=False, download=True, transform=trans)
testloader = torch.utils.data.DataLoader(testset, batch_size=10000, shuffle=False, num_workers=2)

==> Preparing data..


In [6]:
path = "checkpoint/"
def load_net(name,architecture):
    checkpoint = torch.load(path+name+'.t7',map_location='cpu')
    architecture.load_state_dict(checkpoint['net'])
    print(name+' ACC:',checkpoint['acc'])
    return architecture

In [7]:
d=10
name="MNIST"

# Train Softmax

In [18]:
# Model
print('==> Building model..')
classifier = nn.Linear(d, c,bias=True)
net = LeNet(embedding_dim=d, classifier = classifier)
net = net.to(device)

criterion = CE_Loss(classifier, c, device)

sgd = optim.SGD([{'params': net.parameters()},],
                lr=0.1, momentum=0.9, weight_decay=5e-4)
optimizer = Optimizer(sgd, trainloader, device)

==> Building model..


In [19]:
epoch_offset = 0
for lr,max_epochs in [(0.1,40),(0.01,20)]:
    optimizer.optimizer.param_groups[0]['lr'] = lr
    print("===== Optimize with step size ",lr)
    for epoch in range(epoch_offset, epoch_offset+max_epochs):
        print('\nEpoch: %d' % epoch)
        optimizer.train_epoch(net, criterion)
        (acc,conf) = optimizer.test_acc(net,criterion, testloader)
    epoch_offset+= max_epochs

print('Saving..')
state = {
    'net': net.state_dict(),
    'acc': acc
}
torch.save(state, './checkpoint/%s%s%s.t7'%(name,net.__class__.__name__,criterion.classifier.__class__.__name__))


===== Optimize with step size  0.1

Epoch: 0
Loss: 0.378 | Acc: 87.628% (52577/60000) | Conf 86.81 | time (s): 8.09
Loss: 0.113 | Acc: 96.840% (9684/10000) | Conf 96.69

Epoch: 1
Loss: 0.083 | Acc: 97.663% (58598/60000) | Conf 97.63 | time (s): 7.87
Loss: 0.071 | Acc: 97.990% (9799/10000) | Conf 97.58

Epoch: 2
Loss: 0.063 | Acc: 98.192% (58915/60000) | Conf 98.15 | time (s): 7.92
Loss: 0.056 | Acc: 98.230% (9823/10000) | Conf 98.28

Epoch: 3
Loss: 0.058 | Acc: 98.345% (59007/60000) | Conf 98.30 | time (s): 7.99
Loss: 0.064 | Acc: 98.160% (9816/10000) | Conf 98.24

Epoch: 4
Loss: 0.049 | Acc: 98.590% (59154/60000) | Conf 98.51 | time (s): 7.96
Loss: 0.059 | Acc: 98.170% (9817/10000) | Conf 98.63

Epoch: 5
Loss: 0.045 | Acc: 98.663% (59198/60000) | Conf 98.63 | time (s): 8.02
Loss: 0.044 | Acc: 98.740% (9874/10000) | Conf 98.76

Epoch: 6
Loss: 0.042 | Acc: 98.758% (59255/60000) | Conf 98.65 | time (s): 8.11
Loss: 0.074 | Acc: 97.780% (9778/10000) | Conf 98.16

Epoch: 7
Loss: 0.043 | Acc

# Train Gauss

Train Softmax Warmup Model

In [21]:
warmup, epoch_offset = 10, 0
for lr,max_epochs in [(0.1,40),(0.01,20)]:
    optimizer.optimizer.param_groups[0]['lr'] = lr
    print("===== Optimize with step size ",lr)
    for epoch in range(epoch_offset, min(epoch_offset+max_epochs, warmup)):
        print('\nEpoch: %d' % epoch)
        optimizer.train_epoch(net, criterion)
        (acc,conf) = optimizer.test_acc(net,criterion, testloader)
    epoch_offset+= max_epochs

print('Saving..')
state = {
    'net': net.state_dict(),
    'acc': acc
}
torch.save(state, './checkpoint/%s_base%s%s.t7'%(name,net.__class__.__name__,criterion.classifier.__class__.__name__))

===== Optimize with step size  0.1

Epoch: 0
Loss: 0.040 | Acc: 98.743% (59246/60000) | Conf 98.66 | time (s): 7.84
Loss: 0.045 | Acc: 98.650% (9865/10000) | Conf 98.69

Epoch: 1
Loss: 0.032 | Acc: 99.012% (59407/60000) | Conf 98.92 | time (s): 7.97
Loss: 0.042 | Acc: 98.670% (9867/10000) | Conf 99.00

Epoch: 2
Loss: 0.030 | Acc: 99.072% (59443/60000) | Conf 98.98 | time (s): 7.89
Loss: 0.047 | Acc: 98.570% (9857/10000) | Conf 98.73

Epoch: 3
Loss: 0.027 | Acc: 99.178% (59507/60000) | Conf 99.03 | time (s): 7.97
Loss: 0.038 | Acc: 98.900% (9890/10000) | Conf 98.89

Epoch: 4
Loss: 0.029 | Acc: 99.100% (59460/60000) | Conf 99.01 | time (s): 8.04
Loss: 0.040 | Acc: 98.740% (9874/10000) | Conf 98.66

Epoch: 5
Loss: 0.026 | Acc: 99.223% (59534/60000) | Conf 99.07 | time (s): 7.91
Loss: 0.031 | Acc: 99.070% (9907/10000) | Conf 99.15

Epoch: 6
Loss: 0.028 | Acc: 99.173% (59504/60000) | Conf 99.03 | time (s): 8.02
Loss: 0.030 | Acc: 99.060% (9906/10000) | Conf 99.03

Epoch: 7
Loss: 0.028 | Acc

Compute class centroids

In [8]:
optimizer = Optimizer(None, trainloader, device)
net = LeNet(embedding_dim=d, classifier = nn.Linear(d, c,bias=True))
net = load_net('MNIST_baseLeNetLinear',net)
classifier = Gauss(in_features = d, out_features = c, gamma=0.5)
net.classifier = classifier
net = net.to(device)

criterion = CE_GALoss(classifier, c, device)
criterion = criterion.to(device)

optimizer.optimize_centroids(net)
print("Test accuracy after computing class centroids and predicting with Gauss confidence:")
(acc,conf) = optimizer.test_acc(net,criterion, testloader)

state = {
    'net': net.state_dict(),
    'acc': acc
}
torch.save(state, './checkpoint/%s_base%s%s.t7'%(name,net.__class__.__name__,criterion.classifier.__class__.__name__))

MNIST_baseLeNetLinear ACC: tensor(98.9417)
Test accuracy after computing class centroids and predicting with Gauss confidence:
Loss: 6.552 | Acc: 98.600% (9860/10000) | Conf 11.41


Train the Gauss network

In [17]:
# Model
print('==> Building model..')
classifier = Gauss(in_features = d, out_features = c, gamma=1.,gamma_min=0.1)
net = LeNet(embedding_dim=d, classifier = classifier)
net = load_net('MNIST_baseLeNetGauss',net)
net = net.to(device)

criterion = CE_GALoss(classifier, c, device)
criterion=criterion.to(device)

sgd = optim.SGD([
                {'params': net.embed.parameters()},
                {'params': criterion.classifier.weight, 'weight_decay': 0},
                {'params': criterion.classifier.gamma, 'weight_decay': 0},
                {'params': criterion.delta, 'weight_decay': 5e-3}],
                lr=0.1, momentum=0.9, weight_decay=5e-4)
optimizer = Optimizer(sgd, trainloader, device)

==> Building model..
MNIST_baseLeNetGauss ACC: tensor(98.6000)


In [18]:
warmup, epoch_offset = 10, 0
for lr, max_epochs in [(0.1,40),(0.01,20)]:
    lr*=0.2
    for param_group in optimizer.optimizer.param_groups:
        param_group['lr']=lr
    print("==== Optimize with step size ",lr)
    for epoch in range(max(epoch_offset,warmup), epoch_offset+max_epochs):
        print('\nEpoch: %d' % epoch)
        optimizer.train_epoch(net, criterion)
        (acc,conf) = optimizer.test_acc(net,criterion, testloader)
        if epoch%5==0:
            with torch.no_grad():
                margins = criterion.classifier.get_margins()
                gamma = criterion.classifier.gamma
                delta = criterion.delta
                print('Min margin: %.2f, mean margin: %.2f +- %.3f'% (torch.min(margins), torch.mean(margins), torch.std(margins)))
                print('gamma: %.2f +- %.3f'% (torch.mean(gamma), torch.std(gamma)))
                print('delta: %.2f +- %.3f'% (torch.mean(delta), torch.std(delta)))
    epoch_offset+=max_epochs
print('Saving..')
state = {
    'net': net.state_dict(),
    'acc': acc
}
torch.save(state, './checkpoint/%s%s%s.t7'%(name,net.__class__.__name__,criterion.classifier.__class__.__name__))

==== Optimize with step size  0.020000000000000004

Epoch: 10
Loss: 0.456 | Acc: 99.185% (59511/60000) | Conf 64.26 | time (s): 8.48
Loss: 0.334 | Acc: 99.080% (9908/10000) | Conf 70.79
Min margin: 11.64, mean margin: 14.89 +- 1.851
gamma: 0.10 +- 0.003
delta: 0.34 +- 0.042

Epoch: 11
Loss: 0.280 | Acc: 99.390% (59634/60000) | Conf 72.94 | time (s): 8.26
Loss: 0.294 | Acc: 99.140% (9914/10000) | Conf 72.96

Epoch: 12
Loss: 0.239 | Acc: 99.477% (59686/60000) | Conf 76.31 | time (s): 8.31
Loss: 0.276 | Acc: 99.230% (9923/10000) | Conf 76.36

Epoch: 13
Loss: 0.216 | Acc: 99.513% (59708/60000) | Conf 78.40 | time (s): 8.32
Loss: 0.247 | Acc: 99.190% (9919/10000) | Conf 78.65

Epoch: 14
Loss: 0.196 | Acc: 99.587% (59752/60000) | Conf 79.96 | time (s): 8.42
Loss: 0.244 | Acc: 99.220% (9922/10000) | Conf 80.74

Epoch: 15
Loss: 0.190 | Acc: 99.620% (59772/60000) | Conf 80.65 | time (s): 8.81
Loss: 0.241 | Acc: 99.140% (9914/10000) | Conf 79.98
Min margin: 11.49, mean margin: 14.36 +- 1.726
gam

# Train DUQ model

In [12]:
# Model
print('==> Building model..')
classifier = Gauss_DUQ(in_features = d, out_features = c, gamma=1)
net = LeNet(embedding_dim=d, classifier = classifier)
net = net.to(device)

criterion = BCE_DUQLoss(classifier, c, device)
criterion=criterion.to(device)

sgd = optim.SGD([{'params': net.parameters()}],
                lr=0.1, momentum=0.9, weight_decay=5e-4)
optimizer = Optimizer(sgd, trainloader, device, update_centroids = True)

==> Building model..


In [14]:
epoch_offset =0

for lr, max_epochs in [(0.05,25),(0.01,25),(0.002,25)]:
    sgd.param_groups[0]['lr'] = lr
    print("===Optimize with step size ",lr)
    for epoch in range(epoch_offset, epoch_offset+max_epochs):
        print('\nEpoch: %d' % epoch)
        optimizer.train_epoch(net, criterion, weight_gp_pred=1)
        (acc,conf) = optimizer.test_acc(net,criterion, testloader)
    epoch_offset+= max_epochs
    
print('Saving..')
state = {
    'net': net.state_dict(),
    'acc': acc
}
torch.save(state, './checkpoint/%s%s%s.t7'%(name,net.__class__.__name__,criterion.classifier.__class__.__name__))


===Optimize with step size  0.05

Epoch: 0
Loss: 0.299 | Acc: 27.677% (16606/60000) | Conf 22.78 | time (s): 17.47
Loss: 0.223 | Acc: 55.810% (5581/10000) | Conf 46.77

Epoch: 1
Loss: 0.165 | Acc: 75.598% (45359/60000) | Conf 57.48 | time (s): 17.54
Loss: 0.113 | Acc: 83.140% (8314/10000) | Conf 61.85

Epoch: 2
Loss: 0.119 | Acc: 86.622% (51973/60000) | Conf 67.46 | time (s): 17.54
Loss: 0.083 | Acc: 90.200% (9020/10000) | Conf 72.23

Epoch: 3
Loss: 0.095 | Acc: 91.427% (54856/60000) | Conf 72.43 | time (s): 17.42
Loss: 0.072 | Acc: 93.630% (9363/10000) | Conf 71.98

Epoch: 4
Loss: 0.083 | Acc: 93.600% (56160/60000) | Conf 74.69 | time (s): 17.59
Loss: 0.063 | Acc: 94.900% (9490/10000) | Conf 77.01

Epoch: 5
Loss: 0.076 | Acc: 94.988% (56993/60000) | Conf 75.96 | time (s): 17.95
Loss: 0.058 | Acc: 95.710% (9571/10000) | Conf 77.45

Epoch: 6
Loss: 0.071 | Acc: 95.652% (57391/60000) | Conf 77.04 | time (s): 17.57
Loss: 0.055 | Acc: 96.060% (9606/10000) | Conf 78.70

Epoch: 7
Loss: 0.068 

KeyboardInterrupt: 

# Load and Inspect Models

In [29]:
net_sm = LeNet(embedding_dim = d, classifier= nn.Linear(in_features = d, out_features = c))
net_duq = LeNet(embedding_dim = d,classifier= Gauss_DUQ(in_features = d, out_features = c, gamma=1))
net_ga = LeNet(embedding_dim=d, classifier=Gauss(in_features = d, out_features = c, gamma=10))

In [30]:
net_sm = load_net('MNISTLeNetLinear',net_sm).to(device)
net_duq = load_net('MNISTLeNetGauss_DUQ',net_duq).to(device)
net_ga = load_net('MNISTLeNetGauss',net_ga).to(device)

MNISTLeNetLinear ACC: tensor(99.2600)
MNISTLeNetGauss_DUQ ACC: tensor(98.4767)
MNISTLeNetGauss ACC: tensor(99.3600)


In [31]:
criterion_sm = CE_Loss(net_sm.classifier, c, device).to(device)
criterion_ga = CE_GALoss(net_ga.classifier, c, device).to(device)
criterion_duq = BCE_DUQLoss(net_duq.classifier, c, device).to(device)

In [32]:
(acc,conf) = optimizer.test_acc(net_sm,criterion_sm, testloader)
(acc,conf) = optimizer.test_acc(net_duq,criterion_duq, testloader)
(acc,conf) = optimizer.test_acc(net_ga,criterion_ga, testloader)

Loss: 0.024 | Acc: 99.260% (9926/10000) | Conf 99.35
Loss: 0.031 | Acc: 98.420% (9842/10000) | Conf 86.72
Loss: 0.400 | Acc: 99.360% (9936/10000) | Conf 86.90
